In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [4]:
test.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
1,250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
2,250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
3,250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
4,250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

In [11]:
kf = KFold(n_splits = 15, shuffle = False)

In [12]:
skf = StratifiedKFold(n_splits = 15, shuffle = False)

In [46]:
cor = train.corr()['loss']

In [49]:
cor_df = pd.DataFrame({'cor' : cor})

In [51]:
cor_df['abs_cor'] = cor_df['cor'].abs()

In [57]:
use_cols = cor_df[cor_df.abs_cor > 0.005].index.tolist()[:-1]

In [87]:
X = train.iloc[:, 1:-1]
y = train['loss']

target = test.iloc[:, 1:]

In [61]:
scaler = StandardScaler()

In [62]:
scaled_x = scaler.fit_transform(X)

In [63]:
target = scaler.transform(target)

In [64]:
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [29]:
xgb_pred = np.zeros((target.shape[0]))
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    xgb = XGBRegressor(n_estimators = 10000, learning_rate = .02, objective = 'reg:squarederror', random_state = 42)
    
    xgb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 2500, early_stopping_rounds = 1000)
    
    pred = xgb.predict(target) / 15
    
    xgb_pred += pred

[0]	validation_0-rmse:10.06738	validation_1-rmse:10.04094
[2456]	validation_0-rmse:7.15465	validation_1-rmse:7.80603
[0]	validation_0-rmse:10.06930	validation_1-rmse:10.01408
[2471]	validation_0-rmse:7.15024	validation_1-rmse:7.80124
[0]	validation_0-rmse:10.06345	validation_1-rmse:10.09618
[2500]	validation_0-rmse:7.15453	validation_1-rmse:7.86571
[2656]	validation_0-rmse:7.12090	validation_1-rmse:7.86593
[0]	validation_0-rmse:10.06218	validation_1-rmse:10.11556
[2500]	validation_0-rmse:7.13863	validation_1-rmse:7.89171
[3066]	validation_0-rmse:7.02121	validation_1-rmse:7.89484
[0]	validation_0-rmse:10.07364	validation_1-rmse:9.95117
[1897]	validation_0-rmse:7.27428	validation_1-rmse:7.78336
[0]	validation_0-rmse:10.07464	validation_1-rmse:9.93713
[2287]	validation_0-rmse:7.18772	validation_1-rmse:7.75257
[0]	validation_0-rmse:10.06112	validation_1-rmse:10.12936
[1976]	validation_0-rmse:7.24971	validation_1-rmse:7.89150
[0]	validation_0-rmse:10.06504	validation_1-rmse:10.07296
[2226]	

In [83]:
cb.best_score_['validation']['RMSE']

7.849700308999618

In [84]:
cb_pred = np.zeros((target.shape[0]))
cb_rmse = []
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    cb = CatBoostRegressor(silent = True, iterations = 10000, learning_rate = .05, random_state = 42)
    
    train_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb.fit(train_data, eval_set = val_data, early_stopping_rounds = 1000, use_best_model = True, verbose = 2500)
    rmse = cb.best_score_['validation']['RMSE']
    cb_rmse.append(rmse)
    pred = cb.predict(target) / 15
    
    cb_pred += pred
print(f'\nRMSE of CatBoost is {np.mean(cb_rmse)}')

0:	learn: 7.9426871	test: 7.8777472	best: 7.8777472 (0)	total: 18.8ms	remaining: 3m 7s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 7.801766156
bestIteration = 1308

Shrink model to first 1309 iterations.
0:	learn: 7.9413477	test: 7.8968967	best: 7.8968967 (0)	total: 17.4ms	remaining: 2m 54s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 7.802500252
bestIteration = 931

Shrink model to first 932 iterations.
0:	learn: 7.9372369	test: 7.9547850	best: 7.9547850 (0)	total: 18.5ms	remaining: 3m 5s
2500:	learn: 7.3628491	test: 7.8700425	best: 7.8671797 (1728)	total: 44s	remaining: 2m 11s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 7.867179704
bestIteration = 1728

Shrink model to first 1729 iterations.
0:	learn: 7.9352126	test: 7.9827678	best: 7.9827678 (0)	total: 21.7ms	remaining: 3m 36s
2500:	learn: 7.3596966	test: 7.9078431	best: 7.9022439 (1526)	total: 45.6s	remaining: 2m 16s
Stopped by overfitting detector  (1000 iterat

In [80]:
lgbm_pred = np.zeros((target.shape[0]))
lgbm_rmse = []
i = 0
for tr_idx, val_idx in kf.split(X) :
    i += 1
    tr_x, val_x = scaled_x[tr_idx], scaled_x[val_idx]
    tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]
    
    lgbm = LGBMRegressor(n_estimators = 10000, learning_rate = .02, objective = 'l2', random_state = 42)
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 2500, early_stopping_rounds = 1000)
    rmse = np.sqrt(list(lgbm.best_score_['valid_1'].values())[0])
    lgbm_rmse.append(rmse)
    
    pred = lgbm.predict(target) / 15
    
    lgbm_pred += pred
print(f'\nRMSE of LGBM is {np.mean(lgbm_rmse)}')

Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[829]	training's l2: 58.3449	valid_1's l2: 60.9546
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[1264]	training's l2: 56.757	valid_1's l2: 60.9063
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[1075]	training's l2: 57.3932	valid_1's l2: 61.9237
Training until validation scores don't improve for 1000 rounds
[2500]	training's l2: 52.6197	valid_1's l2: 62.4947
Early stopping, best iteration is:
[1613]	training's l2: 55.4837	valid_1's l2: 62.448
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[574]	training's l2: 59.3864	valid_1's l2: 60.577
Training until validation scores don't improve for 1000 rounds
Early stopping, best iteration is:
[1305]	training's l2: 56.6441	valid_1's l2: 60.1915
Training until validation scores don't improve for 10

***

## Make a submission 

In [85]:
submission['loss'] = lgbm_pred * 0.3 + cb_pred * .7

In [86]:
submission.to_csv('2nd.csv', index = False)